베이징의 미세먼지를 예측하기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [7]:
df = pd.read_csv('Downloads/PRSA_data_2010.1.1-2014.12.31.csv',index_col=0)
df.shape

(43824, 12)

In [22]:
df.isna().sum().sum() / len(df) #누락 비율

0.04716593647316539

In [23]:
data = df.dropna() #누락 데이터를 제외

In [29]:
data_one_hot = pd.get_dummies(data) #원핫 인코딩

In [ ]:
# 지난 7일의 기상정보로 다음날의 pm

In [32]:
data_one_hot.head()

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
No,,,,,,,,,,,,,,,
25,2010,1,2,0,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
26,2010,1,2,1,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
27,2010,1,2,2,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
28,2010,1,2,3,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
29,2010,1,2,4,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
